In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

memory = SqliteSaver.from_conn_string(":memory:")


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [3]:
graph = graph_builder.compile(
    checkpointer=memory,
    
    # "tool" in this case is action node
    interrupt_before=["tools"],  
    # Note: can also interrupt __after__ actions, if desired.
    # interrupt_after=["tools"]
)

In [4]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
thread_config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, thread_config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_snOdySm2qizp6cUnTZbig7Vs)
 Call ID: call_snOdySm2qizp6cUnTZbig7Vs
  Args:
    query: LangGraph


# Interrupting

In [5]:
snapshot = graph.get_state(thread_config)
snapshot.next

('tools',)

In [6]:
existing_message = snapshot.values["messages"][-1]
existing_message.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph'},
  'id': 'call_snOdySm2qizp6cUnTZbig7Vs',
  'type': 'tool_call'}]

In [8]:
# `None` will append nothing new to the current state, letting it resume as if it had never been interrupted
events = graph.stream(None, thread_config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://blog.langchain.dev/langgraph-cloud/", "content": "LangGraph is a framework for building agentic and multi-agent applications with LLMs, and LangGraph Cloud is a scalable and reliable infrastructure for deploying them. Learn how LangGraph helps you design, debug, and monitor complex agentic workflows with LLMs."}, {"url": "https://langchain-ai.github.io/langgraph/", "content": "LangGraph is a framework for creating stateful, multi-actor applications with LLMs, using cycles, controllability, and persistence. Learn how to use LangGraph to build agent and multi-agent workflows with examples, features, and integration with LangChain."}]
================================== Ai Message ==================================

Here are some key resources and information about LangGraph:

1. **Overview of LangGraph**:
   - LangGraph is a framework designed for building a